<p><font size="6" color='grey'> <b>

Generative KI. Verstehen. Anwenden. Gestalten.
</b></font> </br></p>

<p><font size="5" color='grey'> <b>
MCP - Model Context Protocol mit LangChain Agent
</b></font> </br></p>

---

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/GenAI.git#subdirectory=04_modul
!uv pip install --system -q fastmcp langchain-mcp-adapters nest_asyncio uvicorn

from genai_lib.utilities import (
    check_environment,
    get_ipinfo,
    setup_api_keys,
    mprint,
    install_packages,
    mermaid,
    get_model_profile,
    extract_thinking,
    load_chat_prompt_template
)

nest_asyncio.apply()  # Wichtig für asyncio in Jupyter

setup_api_keys(['OPENAI_API_KEY'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

# 1 | Intro

---

<p><font color='black' size="5">
Was ist MCP (Model Context Protocol)?
</font></p>

MCP ist ein **standardisiertes Protokoll**, das LLMs (Large Language Models) mit externen Tools und Datenquellen verbindet. MCP ist wie ein **USB für AI** - ein universeller Standard für Tool-Integration.

**Warum MCP?**
- ✅ **Standardisiert**: Ein Protokoll für alle Tools
- ✅ **Wiederverwendbar**: Einmal erstellen, überall nutzen
- ✅ **Skalierbar**: Einfach neue Tools hinzufügen
- ✅ **Production-Ready**: Für echte Anwendungen gedacht

**Architektur:**
```
LangChain Agent
        ↓
MCP Client (langchain-mcp-adapters)
        ↓ HTTP POST (JSON-RPC 2.0)
MCP Server (FastMCP auf localhost:8000)
        ↓
Tools (add, multiply, subtract, divide)
```

<p><font color='black' size="5">
Wie funktioniert MCP?
</font></p>



MCP basiert auf einem **Client-Server-Modell** mit drei Hauptkomponenten:

**1. MCP Server**
- Stellt **Tools** (Funktionen) bereit
- Kann **Ressourcen** (Daten) anbieten
- Implementiert das MCP-Protokoll (JSON-RPC 2.0)
- Läuft als eigenständiger Prozess

**2. MCP Client**
- Verbindet sich mit einem oder mehreren Servern
- Lädt verfügbare Tools vom Server
- Übersetzt LLM-Anfragen in MCP-Protokoll
- Verwaltet die Kommunikation

**3. LLM/Agent**
- Entscheidet, welche Tools benötigt werden
- Orchestriert mehrere Tool-Aufrufe
- Generiert natürlichsprachliche Antworten
- Nutzt Tools über den Client

**Transport-Optionen:**
- `stdio`: Standard Input/Output (für CLI-Tools)
- `streamable_http`: HTTP-basiert (für Web-Services) ✅ In diesem Notebook
- `sse`: Server-Sent Events
- `websocket`: WebSocket-Verbindung

<p><font color='black' size="5">
Praktisches Beispiel: Was passiert?
</font></p>

Ein AI-Assistent hat die Aufgabe: **Berechne (3+5) ✕ 12**

**Ohne MCP:**
```
AI: "Das Ergebnis ist ungefähr 96."
```
→ AI rät oder rechnet im *Kopf* (fehleranfällig)

**Mit MCP:**
```
1. AI: "Ich brauche das add-Tool"
2. MCP Client → Server: add(3, 5)
3. Server → Client: 8
4. AI: "Ich brauche das multiply-Tool"
5. MCP Client → Server: multiply(8, 12)
6. Server → Client: 96
7. AI: "Das Ergebnis ist 96."
```
→ AI nutzt echte Tools (präzise, nachvollziehbar)

**Der Vorteil:**
- ✅ Präzise Ergebnisse (keine Halluzinationen)
- ✅ Erweiterbar (neue Tools einfach hinzufügen)
- ✅ Wiederverwendbar (gleicher Server für viele AIs)
- ✅ Nachvollziehbar (jeder Tool-Call ist dokumentiert)

**Ressourcen:**
- [FastMCP Documentation](https://www.prefect.io/fastmcp)
- [LangChain MCP Adapters](https://github.com/langchain-ai/langchain-mcp-adapters)
- [MCP Official Docs](https://modelcontextprotocol.io/)


# 2 | FastMCP HTTP-Server starten

---

**FastMCP** ist ein Python-Framework, das die Erstellung von MCP-Servern (Model Context Protocol) radikal vereinfacht. Der Name ist angelehnt an FastAPI – die gleiche Philosophie: minimaler Boilerplate-Code (repetitiver, standardisierter Code, der für die Grundstruktur eines Programms nötig ist, aber keine eigentliche Geschäftslogik enthält), maximale Funktionalität.

**Kernidee**

```python
from fastmcp import FastMCP

mcp = FastMCP("MeinServer")

@mcp.tool()
def add(a: int, b: int) -> int:
    """Addiert zwei Zahlen."""
    return a + b
```

Mit nur wenigen Zeilen wird eine Python-Funktion zu einem Tool, das jedes LLM über das MCP-Protokoll aufrufen kann.

**Hauptmerkmale**

| Feature | Beschreibung |
|---------|--------------|
| **Decorator-basiert** | `@mcp.tool()` macht jede Funktion zum MCP-Tool |
| **Automatische Schemas** | Type Hints werden zu JSON-Schemas für das LLM |
| **Mehrere Transports** | HTTP, stdio, SSE, WebSocket |
| **Docstrings → Beschreibungen** | Das LLM "liest" die Docstrings, um Tools zu verstehen |

**Warum FastMCP?**

Ohne FastMCP müsste man das JSON-RPC 2.0 Protokoll, Session-Management und Tool-Schemas manuell implementieren. FastMCP abstrahiert das alles – man schreibt nur die eigentliche Logik.

**Kurz:** FastMCP ist für MCP das, was FastAPI für REST-APIs ist – der schnellste Weg vom Python-Code zum produktionsreifen Server.

<p><font color='black' size="5">
Server erstellen
</font></p>

Es wird ein FastMCP-Server *erstellt*, der dann als **subprocess** gestartet wird.

In [ ]:
#@title 🚀 MCP HTTP-Server erstellen und starten{ display-mode: "form" }

import subprocess
import time
import sys
import os

mprint("### 🚀 MCP HTTP-Server erstellen")
mprint("---")

# SCHRITT 1: Server-Datei erstellen
server_code = '''
"""FastMCP Math Server - HTTP Transport"""
from fastmcp import FastMCP

# FastMCP Server initialisieren
mcp = FastMCP("MathTools")

@mcp.tool()
def add(a: int, b: int) -> int:
    """Addiert zwei Zahlen."""
    return a + b

@mcp.tool()
def multiply(a: int, b: int) -> int:
    """Multipliziert zwei Zahlen."""
    return a * b

@mcp.tool()
def subtract(a: int, b: int) -> int:
    """Subtrahiert b von a."""
    return a - b

@mcp.tool()
def divide(a: float, b: float) -> float:
    """Teilt a durch b."""
    if b == 0:
        raise ValueError("Division durch 0 ist nicht erlaubt")
    return a / b

if __name__ == "__main__":
    # Server mit HTTP Transport starten
    mcp.run(transport="http", host="127.0.0.1", port=8000, path="/mcp")
'''

# Server-Datei schreiben
with open('math_mcp_server.py', 'w', encoding='utf-8') as f:
    f.write(server_code)

print("✅ Server-Datei erstellt: math_mcp_server.py")
print("   • 4 Tools: add, multiply, subtract, divide\n")

# SCHRITT 2: Server als subprocess starten
print("⏳ Starte Server als subprocess...")

# Prüfe ob Server bereits läuft
try:
    import requests
    requests.get("http://127.0.0.1:8000", timeout=1)
    print("⚠️ Server läuft bereits auf Port 8000")
    print("   Bitte führe die Cleanup-Zelle aus oder starte den Kernel neu.\n")
except:
    pass  # Server läuft nicht, gut!

# Server starten
server_process = subprocess.Popen(
    [sys.executable, "math_mcp_server.py"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# Warte bis Server bereit ist
print("⏳ Warte auf Server-Start...")
max_retries = 10
for i in range(max_retries):
    try:
        import requests
        response = requests.post(
            "http://127.0.0.1:8000/mcp",
            json={"jsonrpc": "2.0", "method": "tools/list", "id": 1},
            timeout=2
        )
        if response.status_code in [200, 400, 406]:
            # Server antwortet (auch Fehler = Server läuft)
            break
    except:
        time.sleep(1)
        if i == max_retries - 1:
            print("❌ Server konnte nicht gestartet werden!")
            server_process.kill()
            raise Exception("Server-Start fehlgeschlagen")

mprint("\n✅ MCP-Server läuft auf http://127.0.0.1:8000/mcp")
print(f"💡 Server-Prozess PID: {server_process.pid}")
print("💡 Der Server läuft als echter subprocess!")

<p><font color='black' size="5">
Server-Status testen
</font></p>

Man kann den Server direkt über HTTP testen:

In [ ]:
#@title 🧪 Server-Health-Check{ display-mode: "form" }
import requests

try:
    mprint("### 🧪 Server Health Check (Status)")
    mprint("---")
except NameError:
    print("### 🧪 Server Health Check (Status)")
    print("---")

try:
    # Wir senden einen einfachen Request.
    # Wir erwarten fast schon einen Fehler 400 oder 406,
    # weil wir keine echte Session aufbauen.
    response = requests.get(
        "http://127.0.0.1:8000/mcp",
        timeout=5
    )

    # Prüfung: Lebt der Server?
    # Status 400 (Bad Request) mit MCP-Fehlermeldung ist hier ein GUTES Zeichen!
    # Es bedeutet: Der Server läuft, wartet aber auf einen korrekten Client.

    server_is_alive = False

    if response.status_code == 200:
        server_is_alive = True
    elif response.status_code in [400, 405, 406]:
        # Prüfen, ob die Antwort JSON ist (typisch für MCP)
        if "application/json" in response.headers.get("Content-Type", ""):
            server_is_alive = True

    if server_is_alive:
        print(f"✅ MCP-Server ist ONLINE und erreichbar!")
        print(f"   Status: {response.status_code} (Erwartet)")
        print(f"   Antwort: Der Server hat geantwortet (MCP-Logik läuft).")
        print(f"🌐 URL: http://127.0.0.1:8000/mcp")
        print("\n💡 Man kann jetzt mit dem nächsten Schritt (LangChain Client) fortfahren.")
        print("   Der Client übernimmt das Session-Management automatisch.")
    else:
        print(f"⚠️ Unerwartete Antwort: Status {response.status_code}")
        print(f"   Text: {response.text}")

except requests.exceptions.ConnectionError:
    print("❌ Server nicht erreichbar (Connection Refused).")
    print("   Bitte starten Sie die Zelle 'MCP HTTP-Server erstellen' neu.")
except Exception as e:
    print(f"❌ Fehler: {e}")

# 3 | LangChain MCP Client

---

Der **MCPClient** verbindet sich über **streamable_http** mit dem Server:

In [ ]:
#@title 🔌 MCP Client mit HTTP initialisieren{ display-mode: "form" }

from langchain_mcp_adapters.client import MultiServerMCPClient

mprint("### 🔌 MCP Client Setup (HTTP)")
mprint("---")

# Client konfigurieren mit streamable_http Transport (nicht "http"!)
client = MultiServerMCPClient(
    {
        "math": {
            "transport": "streamable_http",  # ✅ Richtiger Transport-Name!
            "url": "http://127.0.0.1:8000/mcp"  # HTTP-Endpoint
        }
    }
)

print("✅ Client konfiguriert")
print(f"   Transport: streamable_http")
print(f"   Server-URL: http://127.0.0.1:8000/mcp")

<p><font color='black' size="5">
Tools vom Server laden
</font></p>

Der Client lädt jetzt die Tools über **streamable_http** vom Server:

In [ ]:
#@title 🔧 Tools vom Server laden (streamable_http)  { display-mode: "form" }

mprint("### 🔧 Tools vom MCP-Server laden")
mprint("---")

# Tools asynchron laden über streamable_http
tools = await client.get_tools()

print(f"✅ {len(tools)} Tools geladen vom Server:")
for tool in tools:
    print(f"   • {tool.name}: {tool.description}")

mprint("\n💡 Kommunikation erfolgt über streamable_http (JSON-RPC 2.0)!")

# 4 | LangChain Agent

---

<p><font color='black' size="5">
Agent mit MCP-Tools erstellen
</font></p>

Der Agent kommuniziert über **streamable_http** mit dem Server!

In [ ]:
#@title 🤖 LangChain Agent erstellen{ display-mode: "form" }

from langchain.chat_models import init_chat_model
from langchain.agents import create_agent

mprint("### 🤖 LangChain Agent erstellen")
mprint("---")

# SCHRITT 1: LLM initialisieren (LangChain 1.0+ API)
print("1️⃣ LLM initialisieren...")
llm = init_chat_model("openai:gpt-4o-mini", temperature=0.0)
print("   ✅ GPT-4o-mini geladen\n")

# SCHRITT 2: Agent erstellen mit MCP-Tools
print("2️⃣ Agent erstellen...")
agent = create_agent(
    model=llm,
    tools=tools,  # Tools vom MCP-Server (über streamable_http)
    system_prompt="""Du bist ein hilfreicher Mathe-Assistent.
    Du hast Zugriff auf folgende Rechen-Tools über einen MCP HTTP-Server:
    - add: Addiert zwei Zahlen
    - multiply: Multipliziert zwei Zahlen
    - subtract: Subtrahiert zwei Zahlen
    - divide: Teilt zwei Zahlen

    Nutze diese Tools, um komplexe Berechnungen durchzuführen.
    Antworte immer auf Deutsch und erkläre deine Rechenschritte.
    Verwende KEINE LaTeX-Notation wie \\times oder \\div in deinen Antworten.
    Benutze stattdessen einfache Zeichen wie × oder * für Multiplikation.""",
    debug=False  # Weniger Output
)
print("   ✅ Agent bereit\n")

mprint("🎯 Agent ist bereit für Queries!")
mprint("💡 Der Agent kommuniziert über streamable_http mit dem MCP-Server")

# 5 | Praktische Beispiele

---

In [ ]:
#@title 🧜‍♀️ Sequenz-Diagramm { display-mode: "form" }

diagram = '''
sequenceDiagram
    autonumber
    actor User
    participant Agent as LangChain Agent
    participant Client as MCP Client
    participant Server as MCP Server (FastMCP)

    User->>Agent: "Berechne (3+5)*12"
    Agent->>Client: Wählt Tool 'add(3,5)'

    Note over Client,Server: Transport: streamable_http

    Client->>Server: HTTP POST (JSON-RPC)
    activate Server
    Server->>Server: Führt Tool 'add' aus
    Server-->>Client: Return Ergebnis: 8
    deactivate Server

    Client-->>Agent: Ergebnis: 8

    Note right of Agent: Agent entscheidet:<br/>Nächster Schritt nötig

    Agent->>Client: Wählt Tool 'multiply(8,12)'
    Client->>Server: HTTP POST (JSON-RPC)
    activate Server
    Server-->>Client: Return Ergebnis: 96
    deactivate Server

    Client-->>Agent: Ergebnis: 96
    Agent->>User: "Das Ergebnis ist 96"
'''
mermaid(diagram, width=800, height=550)

<p><font color='black' size="5">
Einfache Berechnungen
</font></p>

In [ ]:
#@title 🤖 #1: Addition (3 + 5){ display-mode: "form" }

query = "Berechne 3 + 5"

mprint(f"### 🤖 Query: {query}")
mprint("---")

# Agent aufrufen
response = await agent.ainvoke({
    "messages": [{"role": "user", "content": query}]
})

# Antwort extrahieren
final_message = response["messages"][-1]
mprint(f"\n🤖 **Antwort:** {final_message.content}")

In [ ]:
#@title 🤖 #2: Multiplikation (12 * 24){ display-mode: "form" }

query = "Was ist 12 mal 24?"

mprint(f"### 🤖 Query: {query}")
mprint("---")

response = await agent.ainvoke({
    "messages": [{"role": "user", "content": query}]
})

final_message = response["messages"][-1]
mprint(f"\n🤖 **Antwort:** {final_message.content}")

<p><font color='black' size="5">
Komplexe Berechnungen (Multi-Tool)
</font></p>

Der Agent orchestriert mehrere HTTP-Calls zum Server:

In [ ]:
#@title 🤖 #3: Komplexe Berechnung (3+5) ✕ 12 ✕ 2{ display-mode: "form" }

query = "Berechne (3+5)*12*2 mit den MCP-Mathe-Tools."

mprint(f"### 🤖 Query: {query}")
mprint("---")
print("💡 Erwarteter Ablauf (3 HTTP-Calls zum Server):")
print("   1. HTTP POST: add(3, 5) = 8")
print("   2. HTTP POST: multiply(8, 12) = 96")
print("   3. HTTP POST: multiply(96, 2) = 192\n")

response = await agent.ainvoke({
    "messages": [{"role": "user", "content": query}]
})

final_message = response["messages"][-1]
mprint(f"\n🤖 **Antwort:** {final_message.content}")

In [ ]:
#@title 🤖 #4: Division und Subtraktion{ display-mode: "form" }

query = "Berechne (100 - 20) / 4"

mprint(f"### 🤖 Query: {query}")
mprint("---")
print("💡 Erwarteter Ablauf:")
print("   1. HTTP POST: subtract(100, 20) = 80")
print("   2. HTTP POST: divide(80, 4) = 20.0\n")

response = await agent.ainvoke({
    "messages": [{"role": "user", "content": query}]
})

final_message = response["messages"][-1]
mprint(f"\n🤖 **Antwort:** {final_message.content}")

In [ ]:
#@title 🛑 MCP-Server beenden{ display-mode: "form" }

mprint("### 🛑 MCP-Server Cleanup")
mprint("---")

try:
    # Versuche den subprocess zu beenden
    if 'server_process' in globals():
        server_process.terminate()
        server_process.wait(timeout=5)
        print(f"✅ Server-Prozess (PID {server_process.pid}) wurde beendet")
    else:
        print("⚠️ Keine server_process Variable gefunden")
except Exception as e:
    print(f"⚠️ Fehler beim Beenden: {e}")
    print("💡 Server läuft möglicherweise noch. Kernel neu starten empfohlen.")

# A | Aufgabe
---

<p><font color='black' size="5">
Eigene MCP-Tools hinzufügen
</font></p>

**Ziel:** Erweitere den MCP-Server mit neuen Tools.

**Aufgabe 1: Potenzrechnung**
Füge ein `power(a, b)` Tool hinzu (bearbeite `math_mcp_server.py`, dann Server neu starten).

**Aufgabe 2: Modulo-Operation**
Füge ein `modulo(a, b)` Tool hinzu.

**Aufgabe 3: Komplexe Query**
Teste mit: `"Berechne (5 hoch 2) modulo 7"`

**Schritte:**
1. Bearbeite die Server-Datei `math_mcp_server.py`
2. Starte Server neu (führe Cleanup-Zelle aus, dann Zelle 2 neu)
3. Teste mit eigenen Queries
